In [ ]:
%pip install langchain openai transformers --quiet
%pip install -U langchain-openai --quiet
%pip install torch torchvision torchaudio --quiet
%pip install -U langchain-community --quiet
%pip install ipywidgets --quiet
%pip install pandas --quiet
%pip install scikit-learn --quiet

In [ ]:
import os
import pandas as pd
from PIL import Image as PILImage
from PIL import Image, ImageEnhance, ImageFilter
from transformers import CLIPProcessor, CLIPModel
import torch
from langchain.agents import initialize_agent
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.tools import BaseTool
from sentence_transformers import SentenceTransformer, util
import chromadb
import base64
from chromadb.utils.embedding_functions import OpenCLIPEmbeddingFunction
from chromadb.utils.data_loaders import ImageLoader
from IPython.display import Image, display, Markdown
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


In [ ]:
# Set the OpenAI API key
os.environ["OPENAI_API_KEY"] = "sk-proj-uRoYTQ11Qi4yndbUFWSOT3BlbkFJ5dcSHgZPnJRjSpXUwRU7"

# Initialise ChatGPT
gpt4o = ChatOpenAI(temperature=0, model_name="gpt-4o")

# Initialise CLIP model and processor
clip_model_name = "openai/clip-vit-base-patch32"
clip_processor = CLIPProcessor.from_pretrained(clip_model_name)
clip_model = CLIPModel.from_pretrained(clip_model_name)

local_image_dirs = [
    r'C:\Users\wanghanyu\Desktop\5_Classes\Fabricated',
    r'C:\Users\wanghanyu\Desktop\5_Classes\Forged',
    r'C:\Users\wanghanyu\Desktop\5_Classes\Other',
    r'C:\Users\wanghanyu\Desktop\5_Classes\Sheet Metal',
    r'C:\Users\wanghanyu\Desktop\5_Classes\Turned'
]

In [ ]:
# Image Preprocessing Functions
def preprocess_image(image_path):
    image = PILImage.open(image_path).convert('RGB')

    # Resizing images
    image = image.resize((224, 224))

    # Enhanced contrast
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(2)

    # Denoising
    image = image.filter(ImageFilter.MedianFilter(size=3))

    return image


In [ ]:
# Initialise the output parser
parser = StrOutputParser()

image_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a mechanical engineer specializing in manufacturing processes. Your task is to analyze images of mechanical components, identify their primary manufacturing method, and briefly describe key features of the component."""),

    ("user", [
        {"type": "text", "text": """Analyze the image of the mechanical component, identify its primary manufacturing method, and note key features.

Respond with a concise sentence stating the primary manufacturing method used to produce this component and key features."""},
        {"type": "image_url", "image_url": "data:image/jpeg;base64,{image_data}"},
    ]),
])

# Define the LangChain chain
vision_chain = image_prompt | gpt4o | parser

In [ ]:
def generate_description(image_path):
    
    # Encoding the image
    with open(image_path, 'rb') as image_file:
        image_data = base64.b64encode(image_file.read()).decode('utf-8')
    
    prompt_input = {
        'image_data': image_data
    }

    try:
        response = vision_chain.invoke(prompt_input)
    except Exception as e:
        return f"Error generating description: {e}"
    
    return response

# Define a tool class for describing images
class DescribeImageTool(BaseTool):
    name = "Describe Image Tool"
    description = 'Use this tool to describe images of mechanical parts.'
    
    def _run(self, image_path: str):
        return self.run(image_path)
    
    def run(self, image_path: str):
        description = generate_description(image_path)
        return description

# Initialise tools and agents
tools = [DescribeImageTool()]
agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=gpt4o,
    max_iterations=5,
    early_stopping_method='generate'
)

In [ ]:
def process_directories(directory_paths):
    results = []
    for directory_path in directory_paths:
        missing_files = []
        files_processed = 0
        for filename in os.listdir(directory_path):
            if filename.lower().endswith('.jpg'):
                files_processed += 1
                if files_processed > 50:
                    break
                image_path = os.path.join(directory_path, filename)
                print(f"Processing file: {image_path}")
                if os.path.exists(image_path):
                    try:
                        description = generate_description(image_path)
                        result = {"image_name": filename, "description": description}
                        results.append(result)
                    except Exception as e:
                        results.append({"image_name": filename, "description": f"Error processing file: {str(e)}"})
                else:
                    print(f"File not found: {image_path}")
                    missing_files.append(filename)
                    results.append({"image_name": filename, "description": "File not found"})
        if missing_files:
            print(f"The following files were not found in {directory_path}: {', '.join(missing_files)}")
    
    # Save results to a CSV file
    output_file_path = os.path.join(os.getcwd(), "Non_RAG_01.csv")
    df = pd.DataFrame(results)
    df.to_csv(output_file_path, index=False)
    print(f"CSV file saved at: {output_file_path}")
    
    return results


In [ ]:
# Process the catalogue and generate descriptions
results = process_directories(local_image_dirs)

# Show results
for result in results:
    print(f"{result['image_name']}: {result['description']}")